# Продвинутый Python, семинар 13

**Лектор:** Петров Тимур

**Семинаристы:** Петров Тимур, Коган Александра, Романченко Полина

**Spoiler Alert:** в рамках курса нельзя изучить ни одну из тем от и до досконально (к сожалению, на это требуется больше времени, чем даже 3 часа в неделю). Но мы попробуем рассказать столько, сколько возможно :)

**N.B.** Тут стоит создать свою базу данных на pymongo, но есть нюанс: без VPN оно не открывается :с 

В прошлый раз мы с вами говорили про реляционные СУБД (и в целом он полностью решается через SQLAlchemy, счастье-радость)

Теперь давайте поговорим про нереляционные СУБД (или т.н. NoSQL - Not Only SQL или же No SQL, как кому удобно)

## NoSQL

Что выделяет реляционные СУБД? По сути, структура и логика, как и что должно выглядеть (обычно это называют реляционной алгеброй, учим матчасть).

Можно сразу понять, что есть NoSQL - это попытка избавиться от этих "оков". Где-то это хорошо, а где-то плохо. Киты NoSQL:

* Не используется SQL - ну то есть не используем классический SQL (хотя синтаксис все равно похож)

* SchemaLess структура - нет схемы, следовательно, в любую строку можно добавить любое новое поле без изменения всей таблицы (то есть не нужно отдельно делать ALTER TABLE, но вообще было бы славно такое делать)

* Аггрегационная модель - нет реляционной модели, есть аггрегации! Каждая запись - это все, вместе и сразу, что вам нужно (это не значит, что есть только 1 таблица, но это значит, что если вы те же заказы и платежи храните в разных местах по реляционной БД, то тут можно хранить все вместе)

* Слабые ACID-свойства - в реляционной БД обновление выглядит следующим образом:

    * Я ввожу данные и запускаю скрипт на обновление

    * Таблица лочится для остальных на обновление (на select доступно, но пока внесение данных не закончится, никто их не увидит)

    * Скрипт обновился, лок снялся, данные обновились (достигается атомарность)

В NoSQL же ситуация такая: все вносим, а там разберемся (слабая атомарность транзакций)

Что мы в итоге получаем:

* Реляционная база данных - хороша на чтение (аналитики), хорошая атомарность, но долгое обновление, чистенько-красивенько

* Нереляционная база данных - за счет опускания ограничений мы получаем очень быструю запись и обновление, но структура... хромает

Поэтому так или иначе используется и то, и другое

И сегодня мы поговорим про PyMongo, один из распространенных для Python NoSQL системе

## Создаем себе БД удаленно

**WARNING:** Включите VPN

1. Заходим на MongoDB, регистрируемся

2. Создаем новый кластер (выбираем бесплатный, то есть все по базе, только регион билжайший выбрать лучше, у меня Швеция)

3. Логин-пароль для доступа

4. Дальше Security -> Network Access -> Add IP Address -> 0.0.0.0/0 (доступ для всех IP-адресов)

5. Database -> Кликаем на базу данных, идем в Collections. Можно загрузить Sample Database, что я лично сделал, можно сделать Create Database (там будет название DB и название Collection, что это такое, мы обсудим)

И вуаля! Получится должно примерно вот [так](https://drive.google.com/file/d/1UmpBC5Dld1EYqpoIXiz78S3PsbB57jRg/view?usp=sharing)

6. Тыкаем на Database -> Connect -> Connect your application -> Python, 3.6 or later -> копируем connection string, заменяем пароль и все готово, можно работать

## А теперь к PyMongo!

Что представляет из себя вообще MongoDB как хранилище?

Как вы видите, по сути - это набор опять-таки таблиц и баз данных. Может быть несколько БД, внутри БД может быть несколько таблиц. Таблицы внутри MongoDB называются collections, потому что они устроены немного другим образом. Обращаясь к документации:

```
A collection is a group of documents stored in MongoDB, and can be
thought of as roughly the equivalent of a table in a relational
database.
Getting a collection in PyMongo works the same as getting a database
```

То есть это коллекция документов, которые (подразумевается) выглядят примерно одинаково. Внутри себя документ выглядит по существу как словарь или JSON (поэтому и далее занести данные = занести словарь)

In [1]:
!pip install pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [87]:
import pymongo
from pymongo import MongoClient

cluster = MongoClient('<YOUR LINK>')
db = cluster["sample_mflix"]
collection = db["movies"]

In [26]:
print(cluster.list_database_names())  # какие у нас есть БД
print('-' * 30)
print(db.list_collection_names()) # какие есть коллекции

['sample_airbnb', 'sample_analytics', 'sample_geospatial', 'sample_guides', 'sample_mflix', 'sample_restaurants', 'sample_supplies', 'sample_training', 'sample_weatherdata', 'test', 'admin', 'local']
------------------------------
['movies', 'theaters', 'sessions', 'users', 'comments']


### Поиск

In [31]:
collection.find_one()

{'_id': ObjectId('573a1390f29313caabcd4135'),
 'plot': 'Three men hammer on an anvil and pass a bottle of beer around.',
 'genres': ['Short'],
 'runtime': 1,
 'cast': ['Charles Kayser', 'John Ott'],
 'num_mflix_comments': 0,
 'title': 'Blacksmith Scene',
 'fullplot': 'A stationary camera looks at a large anvil with a blacksmith behind it and one on either side. The smith in the middle draws a heated metal rod from the fire, places it on the anvil, and all three begin a rhythmic hammering. After several blows, the metal goes back in the fire. One smith pulls out a bottle of beer, and they each take a swig. Then, out comes the glowing metal and the hammering resumes.',
 'countries': ['USA'],
 'released': datetime.datetime(1893, 5, 9, 0, 0),
 'directors': ['William K.L. Dickson'],
 'rated': 'UNRATED',
 'awards': {'wins': 1, 'nominations': 0, 'text': '1 win.'},
 'lastupdated': '2015-08-26 00:03:50.133000000',
 'year': 1893,
 'imdb': {'rating': 6.2, 'votes': 1189, 'id': 5},
 'type': 'movie'

Видим, что в словаре может быть почти что угодно. На что стоит обратить внимание - на _id, это автоматически генерируемый ID объекта (и это не строка, но про это позже)

Как устроен поиск внутри? Есть ровно 2 метода:

* find_one() - найди 1 экземпляр

* find() - найди все

In [33]:
collection.find_one({"countries": ['USA']}) # наши фильтры, устроены как поиск в словаре

{'_id': ObjectId('573a1390f29313caabcd4135'),
 'plot': 'Three men hammer on an anvil and pass a bottle of beer around.',
 'genres': ['Short'],
 'runtime': 1,
 'cast': ['Charles Kayser', 'John Ott'],
 'num_mflix_comments': 0,
 'title': 'Blacksmith Scene',
 'fullplot': 'A stationary camera looks at a large anvil with a blacksmith behind it and one on either side. The smith in the middle draws a heated metal rod from the fire, places it on the anvil, and all three begin a rhythmic hammering. After several blows, the metal goes back in the fire. One smith pulls out a bottle of beer, and they each take a swig. Then, out comes the glowing metal and the hammering resumes.',
 'countries': ['USA'],
 'released': datetime.datetime(1893, 5, 9, 0, 0),
 'directors': ['William K.L. Dickson'],
 'rated': 'UNRATED',
 'awards': {'wins': 1, 'nominations': 0, 'text': '1 win.'},
 'lastupdated': '2015-08-26 00:03:50.133000000',
 'year': 1893,
 'imdb': {'rating': 6.2, 'votes': 1189, 'id': 5},
 'type': 'movie'

In [34]:
collection.find({"countries": ['USA']}) # Опа, наш любимый курсор)

In [ ]:
for i in collection.find({"countries": ['Russia']}): #в качестве i вы получаете словари с результатом
    print(i) 

In [40]:
res = collection.find({"countries": ['Russia']})[1]
print(res)

{'_id': ObjectId('573a1399f29313caabcee098'), 'countries': ['Russia'], 'genres': ['Drama'], 'runtime': 92, 'cast': ['Aleksandr Zbruev', 'Mark Goronok', 'Marina Neyolova', 'Mariya Lobachova'], 'title': 'Ty u menya odna', 'lastupdated': '2015-09-07 00:41:48.723000000', 'languages': ['Russian'], 'released': datetime.datetime(1994, 6, 1, 0, 0), 'directors': ['Dmitriy Astrakhan'], 'writers': ['Oleg Danilov'], 'awards': {'wins': 1, 'nominations': 1, 'text': '1 win & 1 nomination.'}, 'year': 1993, 'imdb': {'rating': 6.9, 'votes': 181, 'id': 108421}, 'type': 'movie', 'tomatoes': {'viewer': {'rating': 4.2, 'numReviews': 13, 'meter': 100}, 'lastUpdated': datetime.datetime(2015, 8, 14, 18, 54, 47)}, 'num_mflix_comments': 0}


Поиск внутри PyMongo устроен достаточно просто - передаем словарь, по которому искать, он ищет пересечения, все базово

Но вы можете спросить: а как искать значение по спискам, строке (типа LIKE) и так далее? А вот тут начинаются сложности [синтаксиса](https://www.mongodb.com/docs/manual/reference/operator/query/), в этом смысле SQL более нативный, надо приноровиться



In [47]:
print(len(list(collection.find({"countries": {"$in": ["Russia"]}}))))
print(len(list(collection.find({"countries": ['Russia']}))))

304
198


Что еще из базового мы не покрыли пока что?

* LIMIT

* ORDER BY

* Аггрегации

In [48]:
for i in collection.find({"countries": ['Russia']}).limit(10):
    print(i) 

{'_id': ObjectId('573a1399f29313caabced8e7'), 'countries': ['Russia'], 'genres': ['Documentary'], 'runtime': 60, 'cast': ['Anna Fyodorovna Belova', 'Mikhail Fyodorovich Belov', 'Vasiliy Fyodorovich Belov', 'Sergey Fyodorovich Belov'], 'num_mflix_comments': 0, 'title': 'Belovy', 'lastupdated': '2015-01-10 00:36:44.010000000', 'languages': ['Russian'], 'released': datetime.datetime(2002, 10, 17, 0, 0), 'directors': ['Victor Kossakovsky'], 'writers': ['Victor Kossakovsky'], 'awards': {'wins': 3, 'nominations': 1, 'text': '3 wins & 1 nomination.'}, 'year': 1994, 'imdb': {'rating': 8.2, 'votes': 134, 'id': 106385}, 'type': 'movie', 'tomatoes': {'viewer': {'rating': 4.5, 'numReviews': 127, 'meter': 100}, 'lastUpdated': datetime.datetime(2015, 8, 14, 18, 31, 34)}}
{'_id': ObjectId('573a1399f29313caabcee098'), 'countries': ['Russia'], 'genres': ['Drama'], 'runtime': 92, 'cast': ['Aleksandr Zbruev', 'Mark Goronok', 'Marina Neyolova', 'Mariya Lobachova'], 'title': 'Ty u menya odna', 'lastupdated

In [56]:
for i in collection.find({"countries": ['Russia'], "runtime": {"$exists":True}}).limit(10).sort([("runtime", pymongo.ASCENDING)]):
    print(i.get("runtime"))

3
5
8
10
10
16
20
28
45
45


А теперь отдельно к аггрегации:

Тут нас опять подстерегают сложности с синтаксисом. Так как у нас тут документы, то и аггрегация не выглядят обычным образом. Для того, чтобы сагрегировать, нужно собрать pipeline (то есть буквально сказать, как выполнять запрос)

In [ ]:
res = collection.aggregate([ # Вот это все добро можно занести в три переменные и просто передать
            {"$match": # фильтруем данные
                {
                    "countries": {"$in": ["Russia"]}
                }
            },
            {"$group": # говорим, что группируем
                {
                    "_id": "$genres", # по чему аггрегируем
                    "genre_count": {"$sum": 1} # как назвать поле для группировки и как группировать (сумма +1 - count())
                }
            },
            {
                "$sort": {"genre_count": pymongo.DESCENDING}
            },
            {
                "$limit": 5 
            }
        ])

for i in res:
    print(i)
# Вывод - Россия для грустных

Имеет ли смысл выполнять аггрегации не через PyMongo, а через Python непосредственно? Да, потому что это быстрее (на больших данных ощутимо)

И пара слов про _id: мы с вами видели, что это не строка, а некий ObjectID

In [67]:
collection.find_one({"_id": "573a1390f29313caabcd4135"}) # ничего не дает :с

А как по ним искать? А вот потому что это ObjectID, то по-другому:

In [ ]:
from bson.objectid import ObjectId # https://bsonspec.org/ Binary JSON, используется, чтобы не занимать место

collection.find_one({"_id": ObjectId("573a1390f29313caabcd4135")})

### Вставка/удаление

Одна из главных и приятных фич - это вставка и удаление:

* insert_one - вставить один документ

* insert_many - вставить список документов

* delete_one - удалить один документ (первый по фильтрам)

* delete_many - удалить все документы (опять по фильтрам)

Обратите внимание: тут нет rollback, ту все сразу вставляется и удаляется)

In [70]:
db = cluster["test"]
collection = db["test"]

collection.find_one()

{'_id': 1, 'name': 'Hi'}

In [71]:
new_instance = {"name": "You", "surname": "Capybara"}
collection.insert_one(new_instance)

In [72]:
for i in collection.find():
    print(i)

{'_id': 1, 'name': 'Hi'}
{'_id': ObjectId('6391fdb59b7337962b8bd04d'), 'name': 'You', 'surname': 'Capybara'}


In [74]:
new_instance_2 = {"name": "You", "surname": "UwU"}
new_instance_3 = {"name": "You", "surname": "uWu"}
collection.insert_many([new_instance_2, new_instance_3])
for i in collection.find():
    print(i)

{'_id': 1, 'name': 'Hi'}
{'_id': ObjectId('6391fdb59b7337962b8bd04d'), 'name': 'You', 'surname': 'Capybara'}
{'_id': ObjectId('6391fe0b9b7337962b8bd04f'), 'name': 'You', 'surname': 'UwU'}
{'_id': ObjectId('6391fe0b9b7337962b8bd050'), 'name': 'You', 'surname': 'uWu'}


In [76]:
collection.delete_one({"name": "You"})
for i in collection.find():
    print(i)

{'_id': 1, 'name': 'Hi'}
{'_id': ObjectId('6391fe0b9b7337962b8bd04f'), 'name': 'You', 'surname': 'UwU'}
{'_id': ObjectId('6391fe0b9b7337962b8bd050'), 'name': 'You', 'surname': 'uWu'}


In [78]:
collection.delete_many({"name": "You"})
for i in collection.find():
    print(i)

{'_id': 1, 'name': 'Hi'}


### Join

И на финал: вроде как обсуждали, что JOIN в таких вещах не особо нужен. Но вдруг понадобится, что же делать? А это тоже лежит в аггрегациях в качестве $lookup (знатоки Excel, откликнитесь)

In [89]:
lookup_comments = {
   "$lookup": {
         "from": "comments", # из какой таблицы
         "localField": "_id", # ключ из нашей таблицы
         "foreignField": "movie_id", # ключ из второй таблицы
         "as": "related_comments", # как назвать
   }
}

res = collection.aggregate([ # Вот это все добро можно занести в три переменные и просто передать
            {"$match": # фильтруем данные
                {
                    "countries": {"$in": ["Russia"]}
                }
            },
            lookup_comments,
            {
                "$limit": 5 
            }
        ])

for i in res:
    print(i)

{'_id': ObjectId('573a1398f29313caabceb1fe'), 'plot': "A modern day adaptation of Dostoyevsky's classic novel about a young student who is forever haunted by the murder he has committed.", 'genres': ['Drama'], 'runtime': 126, 'cast': ['Crispin Glover', 'Vanessa Redgrave', 'John Hurt', 'Margot Kidder'], 'poster': 'https://m.media-amazon.com/images/M/MV5BMTI3MDQ2MzEyOV5BMl5BanBnXkFtZTcwNzEwODUzMQ@@._V1_SY1000_SX677_AL_.jpg', 'title': 'Crime and Punishment', 'fullplot': "A modern day adaptation of Dostoyevsky's classic novel about a young student who is forever haunted by the murder he has committed.", 'languages': ['English', 'Polish'], 'released': datetime.datetime(2002, 6, 1, 0, 0), 'directors': ['Menahem Golan'], 'writers': ['Fyodor Dostoevsky (novel)', 'Menahem Golan (adaptation)', 'Menahem Golan (screenplay)'], 'awards': {'wins': 2, 'nominations': 0, 'text': '2 wins.'}, 'lastupdated': '2015-08-13 00:34:02.303000000', 'year': 2002, 'imdb': {'rating': 6.4, 'votes': 463, 'id': 96056}, 

## Животное дня

![](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a3/Caracl_%2801%29%2C_Paris%2C_décembre_2013.jpg/1920px-Caracl_%2801%29%2C_Paris%2C_décembre_2013.jpg)

Это каракал (но в России их называют Шлепами, потому что Шлепа). А это шлепки:

![](https://cdn.fishki.net/upload/post/201505/04/1522103/kotyata-karakala.jpg)

Раньше их причисляли к рысям, но в итоге исследований его выделили в отдельный род каракалов. Обитают преимущественно в Африке и в целом ночные животные. А поскольку их достаточно легко приучить, то они были домашними животными еще в древней Азии (например, в Персии) и в древнем Египте